In [1]:
# !pip install wilds

In [2]:
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
import torchvision.models as models
from wilds.common.data_loaders import get_eval_loader
import torch
from torch import nn
from tqdm import tqdm

In [3]:
# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="camelyon17", download=True)


You can also download the dataset manually at https://wilds.stanford.edu/downloads.


  0%|          | 0/10658709504 [00:00<?, ?Byte/s]

Extracting data/camelyon17_v1.0/archive.tar.gz to data/camelyon17_v1.0

It took 56.87 minutes to download and uncompress the dataset.



In [4]:
BATCH_SIZE = 32
FRACTION = 0.33

# Get the training set
train_data = dataset.get_subset(
    "train",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(train_data)) #302436 initially
# Prepare the standard data loader
train_loader = get_train_loader("standard", train_data, batch_size=BATCH_SIZE)

"""
# (Optional) Load unlabeled data
dataset = get_dataset(dataset="camelyon17", download=True, unlabeled=True)
unlabeled_data = dataset.get_subset(
    "test_unlabeled",
    transform=transforms.Compose(
        [transforms.Resize((448, 448)), transforms.ToTensor()]
    ),
)
unlabeled_loader = get_train_loader("standard", unlabeled_data, batch_size=16)
"""
"""
# Train loop
for labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):
    x, y, metadata = labeled_batch
    unlabeled_x, unlabeled_metadata = unlabeled_batch
    ...
"""

99804


'\n# Train loop\nfor labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):\n    x, y, metadata = labeled_batch\n    unlabeled_x, unlabeled_metadata = unlabeled_batch\n    ...\n'

In [5]:
# Get the test set
id_val_data = dataset.get_subset(
    "id_val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(id_val_data))

# Prepare the evaluation data loader
id_val_loader = get_eval_loader("standard", id_val_data, batch_size=BATCH_SIZE)


11075


In [6]:
# Get the test set
val_data = dataset.get_subset(
    "val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(val_data))

# Prepare the evaluation data loader
val_loader = get_eval_loader("standard", val_data, batch_size=BATCH_SIZE)

11518


In [7]:
# Get the test set
test_data = dataset.get_subset(
    "test",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(test_data))

# Prepare the evaluation data loader
test_loader = get_eval_loader("standard", test_data, batch_size=BATCH_SIZE)

28068


In [8]:

# load the ResNet-18 model, with weights pretrained on ImageNet
resnet18_pretrained = models.resnet18(pretrained=True)

num_params = 0
print("Model's parameters: ")
for n, p in resnet18_pretrained.named_parameters():
    print('\t', n, ': ', p.size())
    num_params += p.numel()
print("Number of model parameters: ", num_params)

"""
formula [(W−K+2P)/S]+1.

W is the input volume
K is the Kernel size
P is the padding
S is the stride

"""

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Model's parameters: 
	 conv1.weight :  torch.Size([64, 3, 7, 7])
	 bn1.weight :  torch.Size([64])
	 bn1.bias :  torch.Size([64])
	 layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn1.weight :  torch.Size([64])
	 layer1.0.bn1.bias :  torch.Size([64])
	 layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn2.weight :  torch.Size([64])
	 layer1.0.bn2.bias :  torch.Size([64])
	 layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn1.weight :  torch.Size([64])
	 layer1.1.bn1.bias :  torch.Size([64])
	 layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn2.weight :  torch.Size([64])
	 layer1.1.bn2.bias :  torch.Size([64])
	 layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
	 layer2.0.bn1.weight :  torch.Size([128])
	 layer2.0.bn1.bias :  torch.Size([128])
	 layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
	 layer2.0.bn2.weight :  torch.Size([128])
	 layer2.0.bn2.bias :  torch.Size([128])
	 layer2.0.downsample.0.weight :  torch.Size([12

'\nformula [(W−K+2P)/S]+1.\n\nW is the input volume\nK is the Kernel size\nP is the padding\nS is the stride\n\n'

In [9]:
# function counting the number of parameters and the number of trainable parameters of a model
# optionally, it will also display the layers
def check_model_parameters(model, display_layers=False):
  num_params = 0
  num_trainable_params = 0
  if display_layers==True:
    print("Model's parameters: ")
  for n, p in model.named_parameters():
      if display_layers == True:
        print('\t', n, ': ', p.size())
      num_params += p.numel()
      if p.requires_grad:
        num_trainable_params += p.numel()
  print("Number of model parameters: ", num_params)
  print("Number of trainable parameters: ", num_trainable_params)

In [10]:
# freeze the model parameters

# check the number of parameters and the number of trainable parameters
check_model_parameters(resnet18_pretrained, display_layers=False)

# freeze all the layers
# for param in resnet18_pretrained.parameters():
  # param.requires_grad = False

# check the number of parameters and the number of trainable parameters
check_model_parameters(resnet18_pretrained, display_layers=False)

Number of model parameters:  11689512
Number of trainable parameters:  11689512
Number of model parameters:  11689512
Number of trainable parameters:  11689512


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 5

In [12]:

resnet18_pretrained.fc = nn.Linear(in_features=512, out_features=2, bias=True)

check_model_parameters(resnet18_pretrained, display_layers=True)

Model's parameters: 
	 conv1.weight :  torch.Size([64, 3, 7, 7])
	 bn1.weight :  torch.Size([64])
	 bn1.bias :  torch.Size([64])
	 layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn1.weight :  torch.Size([64])
	 layer1.0.bn1.bias :  torch.Size([64])
	 layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn2.weight :  torch.Size([64])
	 layer1.0.bn2.bias :  torch.Size([64])
	 layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn1.weight :  torch.Size([64])
	 layer1.1.bn1.bias :  torch.Size([64])
	 layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn2.weight :  torch.Size([64])
	 layer1.1.bn2.bias :  torch.Size([64])
	 layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
	 layer2.0.bn1.weight :  torch.Size([128])
	 layer2.0.bn1.bias :  torch.Size([128])
	 layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
	 layer2.0.bn2.weight :  torch.Size([128])
	 layer2.0.bn2.bias :  torch.Size([128])
	 layer2.0.downsample.0.weight :  torch.Size([12

In [13]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    """
    model: Model object
    train_dataloader: DataLoader over the training dataset
    loss_crt: loss function object
    optimizer: Optimizer object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch training loss, which is an average over the individual batch
       losses
    """
    model.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(train_dataloader)
    for batch_idx, batch in tqdm(enumerate(train_dataloader)):
        # shape: batch_size x 1 x 28 x 28, batch_size x 1
        # print("Train")
        # print(batch)
        batch_img, batch_labels, _ = batch
        # move data to GPU
        batch_img = batch_img.to(device)
        batch_labels = batch_labels.to(device)

        # initialize as zeros all the gradients of the model
        model.zero_grad()

        # get predictions from the FORWARD pass
        # shape: batch_size x 10
        output = model(batch_img)

        loss = loss_crt(output, batch_labels.squeeze())
        loss_scalar = loss.item()

        # BACKPROPAGATE the gradients
        loss.backward()
        # use the gradients to OPTIMISE the model
        optimizer.step()

        epoch_loss += loss_scalar

        pred = output.argmax(dim=1, keepdim=True)
        epoch_accuracy += pred.eq(batch_labels.view_as(pred)).float().mean().item()

    epoch_loss = epoch_loss/num_batches
    epoch_accuracy = 100. * epoch_accuracy/num_batches
    return epoch_loss, epoch_accuracy

def eval_epoch(model, val_dataloader, loss_crt, device):
    """
    model: Model object
    val_dataloader: DataLoader over the validation dataset
    loss_crt: loss function object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch validation loss, which is an average over the individual batch
       losses
    """
    model.eval()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(val_dataloader)
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(val_dataloader)):
            # print("Eval")
            # print(batch)
            # shape: batch_size x 3 x 28 x 28, batch_size x 1
            batch_img, batch_labels, _ = batch
            current_batch_size = batch_img.size(0)

            # move data to GPU
            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device)

            # batch_size x 10
            output = model(batch_img)

            loss = loss_crt(output, batch_labels.squeeze())
            loss_scalar = loss.item()

            epoch_loss += loss_scalar

            pred = output.argmax(dim=1, keepdim=True)
            epoch_accuracy += pred.eq(batch_labels.view_as(pred)).float().mean().item()

    epoch_loss = epoch_loss/num_batches
    epoch_accuracy = 100. * epoch_accuracy/num_batches
    return epoch_loss, epoch_accuracy

In [14]:
resnet18_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(resnet18_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# evaluate the initial model
# val_loss, al_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
# print('Validation performance before finetuning -- loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

# finetune the model
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(resnet18_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

3119it [10:05,  5.15it/s]
347it [00:40,  8.65it/s]



Epoch 1
train loss: 0.11830959, accuracy: 95.94922251
id_val loss: 0.09538839, accuracy: 96.34365994


3119it [06:55,  7.51it/s]
347it [00:27, 12.53it/s]



Epoch 2
train loss: 0.05602810, accuracy: 98.00917762
id_val loss: 0.05096812, accuracy: 98.11779539


3119it [06:21,  8.17it/s]
347it [00:27, 12.83it/s]



Epoch 3
train loss: 0.03772924, accuracy: 98.70050497
id_val loss: 0.04866841, accuracy: 98.24387608


3119it [06:22,  8.16it/s]
347it [00:27, 12.46it/s]



Epoch 4
train loss: 0.02781835, accuracy: 99.04402166
id_val loss: 0.05344228, accuracy: 98.29791066


3119it [06:25,  8.09it/s]
347it [00:27, 12.62it/s]


Epoch 5
train loss: 0.02083950, accuracy: 99.25743141
id_val loss: 0.05884800, accuracy: 97.99171470


In [15]:
def run_eval_ood(model, loader, loss_criterion, device, eval_type):
  losses = []
  accuracies = []

  loss, accuracy = eval_epoch(model, loader, loss_criterion, device)
  losses.append(loss)
  accuracies.append(accuracy)
  print(eval_type + ' loss: %10.8f, accuracy: %10.8f'%(loss, accuracy))

In [16]:
run_eval_ood(resnet18_pretrained, val_loader, loss_criterion, device, "val")

360it [00:43,  8.24it/s]

val loss: 0.46796423, accuracy: 87.63715279


In [17]:
run_eval_ood(resnet18_pretrained, test_loader, loss_criterion, device, "test")

878it [01:48,  8.10it/s]

test loss: 1.12270584, accuracy: 77.62314920


In [18]:
densenet121_pretrained = models.densenet121(pretrained=True)

# for param in densenet121_pretrained.parameters():
  # param.requires_grad = False

check_model_parameters(densenet121_pretrained, display_layers=False)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

Number of model parameters:  7978856
Number of trainable parameters:  7978856


In [19]:

densenet121_pretrained.classifier = nn.Linear(in_features=1024, out_features=2, bias=True)
check_model_parameters(densenet121_pretrained, display_layers=False)

Number of model parameters:  6955906
Number of trainable parameters:  6955906


In [20]:
densenet121_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(densenet121_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# evaluate the initial model
# val_loss, al_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
# print('Validation performance before finetuning -- loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

# finetune the model
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
id_val_losses = []
id_val_accuracies = []

for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(densenet121_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(densenet121_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


3119it [15:59,  3.25it/s]
347it [00:48,  7.20it/s]



Epoch 1
train loss: 0.08947193, accuracy: 96.96717698
id_val loss: 0.04903783, accuracy: 98.27089337


3119it [15:52,  3.28it/s]
347it [00:47,  7.36it/s]



Epoch 2
train loss: 0.04349755, accuracy: 98.54520680
id_val loss: 0.03753684, accuracy: 98.72118156


3119it [15:50,  3.28it/s]
347it [00:47,  7.33it/s]



Epoch 3
train loss: 0.03116413, accuracy: 98.91276966
id_val loss: 0.03892054, accuracy: 98.68515850


3119it [15:50,  3.28it/s]
347it [00:46,  7.48it/s]



Epoch 4
train loss: 0.02413429, accuracy: 99.13834562
id_val loss: 0.04475565, accuracy: 98.57708934


3119it [15:48,  3.29it/s]
347it [00:47,  7.32it/s]


Epoch 5
train loss: 0.02237113, accuracy: 99.20547451
id_val loss: 0.04431844, accuracy: 98.48703170


In [21]:
run_eval_ood(densenet121_pretrained, val_loader, loss_criterion, device, "val")

360it [00:49,  7.26it/s]

val loss: 0.61660476, accuracy: 85.78125000


In [22]:
run_eval_ood(densenet121_pretrained, test_loader, loss_criterion, device, "test")

878it [02:00,  7.29it/s]

test loss: 0.91136280, accuracy: 79.53445330


In [23]:
torch.save(resnet18_pretrained.state_dict(), "resnet18_pretrained.pt")

In [24]:
torch.save(densenet121_pretrained.state_dict(), "densenet121_pretrained.pt")

In [25]:
resnetxt50_pretrained = models.resnext50_32x4d(pretrained=True)

# freeze all the layers
# for param in resnetxt50_pretrained.parameters():
  # param.requires_grad = False

resnetxt50_pretrained.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

resnetxt50_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(resnetxt50_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()


# finetune the model
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(resnetxt50_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(resnetxt50_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

3119it [17:44,  2.93it/s]
347it [00:43,  7.91it/s]



Epoch 1
train loss: 0.08606202, accuracy: 97.03115696
id_val loss: 0.05096113, accuracy: 98.17182997


3119it [17:43,  2.93it/s]
347it [00:43,  7.97it/s]


...


Epoch 3
train loss: 0.02918555, accuracy: 98.98104360
id_val loss: 0.03763097, accuracy: 98.68515850


3119it [17:42,  2.93it/s]
347it [00:44,  7.88it/s]



Epoch 4
train loss: 0.02205006, accuracy: 99.21048413
id_val loss: 0.03875422, accuracy: 98.63112392


3119it [17:44,  2.93it/s]
347it [00:43,  7.97it/s]


Epoch 5
train loss: 0.01629151, accuracy: 99.41788233
id_val loss: 0.04178602, accuracy: 98.77521614


In [26]:
run_eval_ood(resnetxt50_pretrained, val_loader, loss_criterion, device, "val")

360it [00:45,  7.89it/s]

val loss: 0.40985703, accuracy: 87.87210650


In [27]:
run_eval_ood(resnetxt50_pretrained, test_loader, loss_criterion, device, "test")

878it [01:51,  7.87it/s]

test loss: 1.72625949, accuracy: 72.21668565


In [28]:
torch.save(densenet121_pretrained.state_dict(), "resnetxt50_pretrained.pt")

In [29]:
"""
Rupere cap de clasificare si calculare distanta l2 si cos medie intre ood si iid
"""
"""
modules = list(resnet18_pretrained.children())[:-1]
resnet18_pretrained = nn.Sequential(*modules)

average_l2_iid = [0, 0]
model.eval()
num_batches = len(val_dataloader)
with torch.no_grad():
    for batch_idx, batch in tqdm(enumerate(val_dataloader)):
        batch_img, batch_labels, _ = batch
        current_batch_size = batch_img.size(0)

        batch_img = batch_img.to(device)
        batch_labels = batch_labels.to(device)

        output = model(batch_img)
"""

'\nmodules = list(resnet18_pretrained.children())[:-1]\nresnet18_pretrained = nn.Sequential(*modules)\n\naverage_l2_iid = [0, 0]\nmodel.eval()\nnum_batches = len(val_dataloader)\nwith torch.no_grad():\n    for batch_idx, batch in tqdm(enumerate(val_dataloader)):\n        batch_img, batch_labels, _ = batch\n        current_batch_size = batch_img.size(0)\n\n        batch_img = batch_img.to(device)\n        batch_labels = batch_labels.to(device)\n\n        output = model(batch_img)\n'